# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 8 2023 1:18PM,258952,10,Yusen-9244357,Yusen – 3D Matrix,Released
1,Mar 8 2023 1:08PM,258951,10,PRF-55030,Bio-PRF,Released
2,Mar 8 2023 1:08PM,258951,10,PRF-55034,Bio-PRF,Released
3,Mar 8 2023 1:08PM,258951,10,PRF-55039,Bio-PRF,Released
4,Mar 8 2023 1:06PM,258950,19,N17988,"NAPP Technologies, LLC",Released
5,Mar 8 2023 1:01PM,258948,10,Clear-9244344,"ClearSpec, LLC",Released
6,Mar 8 2023 12:30PM,258946,19,ADV80013338,"AdvaGen Pharma, Ltd",Released
7,Mar 8 2023 12:30PM,258946,19,ADV80013340,"AdvaGen Pharma, Ltd",Released
8,Mar 8 2023 12:30PM,258946,19,ADV80013341,"AdvaGen Pharma, Ltd",Released
9,Mar 8 2023 12:30PM,258946,19,ADV80013354,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,258946,Released,6
19,258948,Released,1
20,258950,Released,1
21,258951,Released,3
22,258952,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258946,NaN,NaN,6.0
258948,NaN,NaN,1.0
258950,NaN,NaN,1.0
258951,NaN,NaN,3.0
258952,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258759,21.0,7.0,3.0
258780,2.0,14.0,0.0
258886,0.0,1.0,0.0
258896,0.0,0.0,5.0
258897,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258759,21,7,3
258780,2,14,0
258886,0,1,0
258896,0,0,5
258897,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258759,21,7,3
1,258780,2,14,0
2,258886,0,1,0
3,258896,0,0,5
4,258897,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258759,21,7,3
1,258780,2,14,
2,258886,,1,
3,258896,,,5
4,258897,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 8 2023 1:18PM,258952,10,Yusen – 3D Matrix
1,Mar 8 2023 1:08PM,258951,10,Bio-PRF
4,Mar 8 2023 1:06PM,258950,19,"NAPP Technologies, LLC"
5,Mar 8 2023 1:01PM,258948,10,"ClearSpec, LLC"
6,Mar 8 2023 12:30PM,258946,19,"AdvaGen Pharma, Ltd"
12,Mar 8 2023 12:17PM,258944,10,"Nextsource Biotechnology, LLC"
15,Mar 8 2023 11:32AM,258941,15,"Brookfield Pharmaceuticals, LLC"
30,Mar 8 2023 11:18AM,258940,20,"HVL, LLC dba Atrium Innovations"
31,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc."
35,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 8 2023 1:18PM,258952,10,Yusen – 3D Matrix,,,1
1,Mar 8 2023 1:08PM,258951,10,Bio-PRF,,,3
2,Mar 8 2023 1:06PM,258950,19,"NAPP Technologies, LLC",,,1
3,Mar 8 2023 1:01PM,258948,10,"ClearSpec, LLC",,,1
4,Mar 8 2023 12:30PM,258946,19,"AdvaGen Pharma, Ltd",,,6
5,Mar 8 2023 12:17PM,258944,10,"Nextsource Biotechnology, LLC",,,3
6,Mar 8 2023 11:32AM,258941,15,"Brookfield Pharmaceuticals, LLC",,,15
7,Mar 8 2023 11:18AM,258940,20,"HVL, LLC dba Atrium Innovations",,1,
8,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",,,4
9,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 1:18PM,258952,10,Yusen – 3D Matrix,1,,
1,Mar 8 2023 1:08PM,258951,10,Bio-PRF,3,,
2,Mar 8 2023 1:06PM,258950,19,"NAPP Technologies, LLC",1,,
3,Mar 8 2023 1:01PM,258948,10,"ClearSpec, LLC",1,,
4,Mar 8 2023 12:30PM,258946,19,"AdvaGen Pharma, Ltd",6,,
5,Mar 8 2023 12:17PM,258944,10,"Nextsource Biotechnology, LLC",3,,
6,Mar 8 2023 11:32AM,258941,15,"Brookfield Pharmaceuticals, LLC",15,,
7,Mar 8 2023 11:18AM,258940,20,"HVL, LLC dba Atrium Innovations",,1,
8,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4,,
9,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 1:18PM,258952,10,Yusen – 3D Matrix,1,,
1,Mar 8 2023 1:08PM,258951,10,Bio-PRF,3,,
2,Mar 8 2023 1:06PM,258950,19,"NAPP Technologies, LLC",1,,
3,Mar 8 2023 1:01PM,258948,10,"ClearSpec, LLC",1,,
4,Mar 8 2023 12:30PM,258946,19,"AdvaGen Pharma, Ltd",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 1:18PM,258952,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Mar 8 2023 1:08PM,258951,10,Bio-PRF,3.0,NaN,NaN
2,Mar 8 2023 1:06PM,258950,19,"NAPP Technologies, LLC",1.0,NaN,NaN
3,Mar 8 2023 1:01PM,258948,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Mar 8 2023 12:30PM,258946,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 1:18PM,258952,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Mar 8 2023 1:08PM,258951,10,Bio-PRF,3.0,0.0,0.0
2,Mar 8 2023 1:06PM,258950,19,"NAPP Technologies, LLC",1.0,0.0,0.0
3,Mar 8 2023 1:01PM,258948,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Mar 8 2023 12:30PM,258946,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1553659,11.0,2.0,0.0
12,776708,1.0,2.0,0.0
15,1294273,27.0,21.0,23.0
19,776807,9.0,0.0,0.0
20,258940,0.0,1.0,0.0
21,258904,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1553659,11.0,2.0,0.0
1,12,776708,1.0,2.0,0.0
2,15,1294273,27.0,21.0,23.0
3,19,776807,9.0,0.0,0.0
4,20,258940,0.0,1.0,0.0
5,21,258904,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,11.0,2.0,0.0
1,12,1.0,2.0,0.0
2,15,27.0,21.0,23.0
3,19,9.0,0.0,0.0
4,20,0.0,1.0,0.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,11.0
1,12,Released,1.0
2,15,Released,27.0
3,19,Released,9.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,23.0,0.0,0.0,0.0
Executing,2.0,2.0,21.0,0.0,1.0,1.0
Released,11.0,1.0,27.0,9.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,23.0,0.0,0.0,0.0
1,Executing,2.0,2.0,21.0,0.0,1.0,1.0
2,Released,11.0,1.0,27.0,9.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,23.0,0.0,0.0,0.0
1,Executing,2.0,2.0,21.0,0.0,1.0,1.0
2,Released,11.0,1.0,27.0,9.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()